In [1]:
import pandas as pd 
import numpy as np
import time

# # Solución 1: La Dieta de Columnas (`usecols`)

In [2]:
#1. Creamos el dataframe
df_gordo=pd.DataFrame({
    'fecha': ['2024-01-01', '2024-01-02', '2024-01-03'] * 15000,
    'ip': ['192.168.1.1', '127.0.0.1', '10.0.0.1']* 15000,
    'usuario': ['Admin', 'Guest', 'Root'] * 15000,
    'mensaje': ['Error 404', 'Login OK', 'Timeout']*15000
})

#2. Guardamos el archivo
df_gordo.to_csv('Archivos/log_servidor.csv', index=False)

df_gordo.to_pickle('Archivos/log_servidor.pkl')

#3. Medir uso de memoria
print(f"Uso de memoria: {df_gordo.memory_usage(deep=True).sum()} bytes")

Uso de memoria: 2880132 bytes


In [3]:
#4. Crear una "deposito" nuevo para el archivo y extraer solo las columnas 'fecha' y 'mensaje'

#Optimizar los datos de las columnas
columnas_optimizadas = {
    'mensaje' : 'category'  #Optimización de mensajes
}
df_flaco = pd.read_csv('Archivos/log_servidor.csv',
                    usecols=['fecha','mensaje'],  #Filtrado vertical
                    dtype=columnas_optimizadas,
                    parse_dates=['fecha'],  #Parseo temporal inmediato
                    index_col='fecha'
)

print(df_flaco.info())
print("\nObserva: El índice ya es Datetime y 'mensaje' es Category.")

<class 'pandas.DataFrame'>
DatetimeIndex: 45000 entries, 2024-01-01 to 2024-01-03
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   mensaje  45000 non-null  category
dtypes: category(1)
memory usage: 395.6 KB
None

Observa: El índice ya es Datetime y 'mensaje' es Category.


In [4]:
#Uso de memoria de bytes a megabytes
df_gordo_mb = df_gordo.memory_usage(deep=True).sum() / (1024 ** 2)
df_flaco_mb = df_flaco.memory_usage(deep=True).sum() / (1024 ** 2)

print(f"df_gordo: {df_gordo_mb:.2f} MB")
print(f"df_flaco: {df_flaco_mb:.2f} MB")


df_gordo: 2.75 MB
df_flaco: 0.39 MB


# Solución 2: El Traductor Anticipado (`dtype` & `parse_dates`)

In [5]:
df_sucursal = pd.DataFrame ({
    'fecha':['2024-01-01','2024-02-01'],
    'sucursal_id' :['001','002'],
    'tipo_tienda' : ['A','B']
})

#Crear archivo sucursales.csv
df_sucursal.to_csv('Archivos/sucursales.csv', index=False)

#Definir tipos
tipos = {
    'sucursal_id': 'object',    # texto
    'tipo_tienda': 'category'   # categoría
}

#Leer CSV y añadir dtype
df_read = pd.read_csv(
    'Archivos/sucursales.csv',             # reemplaza con tu ruta
    dtype=tipos,               # aplica los tipos del diccionario
    parse_dates=['fecha']      # convierte la columna fecha a datetime
)

In [6]:
#Imprimir información
print("--------- Información DtaFrame de sucursales----------")
print(df_read.info())

--------- Información DtaFrame de sucursales----------
<class 'pandas.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   fecha        2 non-null      datetime64[us]
 1   sucursal_id  2 non-null      object        
 2   tipo_tienda  2 non-null      category      
dtypes: category(1), datetime64[us](1), object(1)
memory usage: 184.0+ bytes
None


# Solución 3: El Desfile de Hormigas (`chunksize`)

In [7]:
#Crear DataFrame
df_data=pd.DataFrame({
    'valores':np.arange(1,11)
})

#Crear Archivo CSV
df_data.to_csv('Archivos/mini_bigdata.csv', index=False)

#Crear iterador y chucksize
chucksize_data=pd.read_csv("Archivos/mini_bigdata.csv", chunksize=2)
suma_total=0
for i, lote_mini in enumerate (chucksize_data):
    suma_lote=lote_mini["valores"].sum()
    suma_total+=suma_lote
    print(f"Lote {i+1}: Suma Parcial:{suma_lote}")


print(f"\Resultado Final:{suma_total}")


Lote 1: Suma Parcial:3
Lote 2: Suma Parcial:7
Lote 3: Suma Parcial:11
Lote 4: Suma Parcial:15
Lote 5: Suma Parcial:19
\Resultado Final:55


<>:18: SyntaxWarning: "\R" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\R"? A raw string is also an option.
<>:18: SyntaxWarning: "\R" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\R"? A raw string is also an option.
C:\Users\danny\AppData\Local\Temp\ipykernel_8404\638930373.py:18: SyntaxWarning: "\R" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\R"? A raw string is also an option.
  print(f"\Resultado Final:{suma_total}")


# Solución 4: La Carrera de Formatos (Pickle vs CSV)

In [8]:
df_person = pd.DataFrame({
    "id": [1, 2, 3, 4],
    "nombre": ["Ana", "Luis", "María", "Juan"],
    "edad": [23, 30, 27, 35]
})

#Crear Archivos csv y pkl
try:
    df_person.to_csv('Archivos/datos.csv',index=False)
    df_person.to_pickle('Archivos/datos.pkl')
    print("Archivos Creados exitosamente")
except:
    print('No se pudo crear los archivos, intente de nuevo')

print("---------TIEMPO DE carga de archivos-----")
# Iniciar medición de tiempo
inicio_csv = time.time()
# Cargar el CSV
df_time = pd.read_csv('Archivos/datos.csv')
fin_csv = time.time()
tiempo_csv= fin_csv-inicio_csv
print(f"Tiempo CSV: {tiempo_csv:.4f} segundos")

# Iniciar medición de tiempo
inicio_pkl = time.time()
# Cargar el CSV
df_pkl = pd.read_pickle('Archivos/datos.pkl')
fin_pkl = time.time()
tiempo_pkl= fin_pkl-inicio_pkl
print(f"Tiempo Pickle: {tiempo_pkl:.4f} segundos")
print(f"Pickle es {tiempo_csv / tiempo_pkl:.1f} veces más rápido.")




Archivos Creados exitosamente
---------TIEMPO DE carga de archivos-----
Tiempo CSV: 0.0043 segundos
Tiempo Pickle: 0.0029 segundos
Pickle es 1.5 veces más rápido.


In [9]:
#Cuanto tarda en cargarse CSV
